```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)
class : int
value : str
var : str                 # non-terminal name
definition : list(rule)
rule : list(var | token)  # right side of the rule
```



The algorithm for filtering out useless nonterminals is the following:
If the nonterminal `A` is useful, then production from `A` must produce a string consisting only of terminals in a finite number of steps.
The production is finite if it consists only of:
1. Terminals, or
2. Useful nonterminals.

In [9]:
def get_useful_var(useful_nonterminals, tokens, vars):
  for var, rules in vars.items():
    if var in useful_nonterminals:
      continue
    for rule in rules:
      # if every symbol in the rule is either a terminal or useful non terminal,
      # then var is also useful non terminal
      if all(map(lambda x: x in tokens or x in useful_nonterminals, rule)):
        return var
  return None

def get_useful_nonterminals(grammar):
  tokens = grammar['toks']
  vars = grammar['vars']

  useful_nonterminals = set()

  while True:
    uv = get_useful_var(useful_nonterminals, tokens, vars)
    if uv == None:
      return useful_nonterminals
    useful_nonterminals.add(uv)


To verify that filtering function works, running it on a sample grammar, which
nonterminals are `S, A, E`

In [13]:
GRAMMAR = {
  'toks': set([
      (0, 'a'),
      (1, 'b'),
  ]),
  'vars': {
      'S': [['A', (0, 'a')], [(1, 'b'), 'S', 'B']],
      'A': [['B', 'A'], ['E', 'S'], [(1, 'b'), 'E']],
      'B': [[(0, 'a'), (1, 'b'), 'D'], [(1, 'b'), 'B']],
      'C': [['B', 'A'], [(1, 'b'), 'C']],
      'D': [['D', 'B'], [(0, 'a'), 'D']],
      'E': [['A', 'S'], [(1, 'b')], ['F']],
      'F': [[]]
  },
  'hvar': 'S'
}

print(get_useful_nonterminals(GRAMMAR))

{'A', 'S', 'F', 'E'}


Now we need to remove those rules, which have useless nonterminals. This is straightforward - every symbol in the rule must be either a terminal, or useful nonterminal.

In [12]:
def reduced_grammar(grammar):
  nonterminals = get_useful_nonterminals(grammar)
  
  tokens = grammar['toks']
  vars = grammar['vars']

  new_vars = {}
  for var, rules in vars.items():
    if var not in nonterminals:
      continue
    new_vars[var] = []
    for rule in rules:
      valid_rule = all(map(lambda x: x in tokens or x in nonterminals, rule))
      if valid_rule:
        new_vars[var].append(rule)
  return {
      'toks': tokens,
      'vars': new_vars,
      'hvar': grammar['hvar']
  }
    
print(reduced_grammar(GRAMMAR))

{'toks': {(1, 'b'), (0, 'a')}, 'vars': {'S': [['A', (0, 'a')]], 'A': [['E', 'S'], [(1, 'b'), 'E']], 'E': [['A', 'S'], [(1, 'b')]]}, 'hvar': 'S'}


Nonterminal is disappearing if all productions from it are consisting of disappearing nonterminals. Obviously, this is also the case when production is empty set. Otherwise, grammar doesn't have disappearing nonterminals.

In [14]:
def get_disappearing_nonterminal(dnts, tokens, vars):
  for var, rules in vars.items():
    if var in dnts:
      continue
    for rule in rules:
      # if every symbol in the rule is either a terminal or useful non terminal,
      # then var is also useful non terminal
      if all(map(lambda x: x in dnts, rule)):
        return var
  return None

def disappearing_nonterminals(grammar):
  tokens = grammar['toks']
  vars = grammar['vars']

  disappearing_nonterminals = set()

  while True:
    dnt = get_disappearing_nonterminal(disappearing_nonterminals, tokens, vars)
    if dnt == None:
      return disappearing_nonterminals
    disappearing_nonterminals.add(dnt)

print(disappearing_nonterminals(GRAMMAR))

{'F', 'E'}
